In [1]:
import pandas as pd
import os
import sys
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import shutil
from osgeo import ogr
from collections import OrderedDict
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import MaxNLocator
import boto3


sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import pull_csv_from_directory, upload_csv_aws, delete_items
from scripts.utils.cal_crai_calculations import calculate_equal_weighted_index, add_census_tracts
from scripts.utils.cal_crai_plotting import index_plot, plot_region_domain, plot_hazard_score

In [2]:
# read in CA census tiger file
ca_tract_county = "s3://ca-climate-index/0_map_data/ca_tracts_county.csv"
ca_tract_county = gpd.read_file(ca_tract_county)
ca_tract_county = ca_tract_county.drop(columns={'field_1', 'geometry', 'COUNTYFP'})
ca_tract_county = ca_tract_county.rename(columns={'TRACT':'GEOID'})

In [3]:
# pull csv from aws
bucket_name = 'ca-climate-index'
aws_dir = '3_fair_data/index_data/calcrai_score'
output_folder = 'cal_crai_score_file'

pull_csv_from_directory(bucket_name, aws_dir, output_folder, search_zipped=False, print_name=False)

Metric data retrieved from 3_fair_data/index_data/calcrai_score.


In [4]:
calcrai_score = pd.read_csv(r'cal_crai_score_file/calcrai_score.csv')

In [5]:
# Initialize the S3 client
s3_client = boto3.client('s3')

# Bucket name and file paths
bucket_name = 'ca-climate-index'
directory = '0_map_data/crosswalk_data/final_cci_project_indicators_and_climate_risk_with_contextual_columns.csv'

print('Pulling file')
s3_client.download_file(bucket_name, directory, 'final_cci_project_indicators_and_climate_risk_with_contextual_columns.csv')
print('File pulled')

Pulling file
File pulled


In [6]:
crosswalk_data = pd.read_csv('final_cci_project_indicators_and_climate_risk_with_contextual_columns.csv')

C:\Users\jesse\AppData\Local\Temp\ipykernel_23820\1874589849.py:1: DtypeWarning: Columns (1,7,8,14,15,16,17,21,25,29,53,66,110,112,116,117,127,128,129,133,134,135) have mixed types. Specify dtype option on import or set low_memory=False.
  crosswalk_data = pd.read_csv('final_cci_project_indicators_and_climate_risk_with_contextual_columns.csv')


In [7]:
climate_mitigation_cci_data = crosswalk_data[pd.notna(crosswalk_data['Climate_Risk_Mitigation'])]

In [8]:
for col in climate_mitigation_cci_data.columns:
    print(col)

Project IDNumber
Reporting Cycle Name
Agency Name
Program Name
Program Description
Sub Program Name
Record Type
Project Name
Project Type
Project Description
SECTOR
CATEGORY
ACTION
Census Tract
Address
Lat Long
Senate
District
Assembly
District
County
Total Project Cost
Total Program GGRFFunding
Project Life Years
Total Project GHGReductions
Annual Project GHGReductions
Project Count
Fiscal Year Funding Project
Is Benefit Disadvantaged Communities
Disadvantaged Community Criteria
Disadvantaged Community Need
Disadvantaged Community Census Tracts
Total GGRFDisadvantaged Community Funding
Disadvantaged Community Benefits Description
Funding Benefiting Disadvantaged Communities
Estimated Num Vehicles In Service
Funding Within Disadvantage Communities
Other Project Benefits Description
VMTReductions
Number Of Housing Units
Number Of Affordable Housing Units
Estimated Number Of Trees To Be Planted
Energy Cost Savings
Estimated Energy Saved KWH
Estimated Energy Saved Therms
Estimated Water S

In [9]:
spatial_columns = ['Lat Long', 'Census Tract', 'County']

# Select only the specified spatial columns
inspect_spatial_columns = climate_mitigation_cci_data[spatial_columns]

# Display the count of non-NaN values for each column
print("Non-NaN value counts for selected spatial columns:")
for column in spatial_columns:
    non_nan_count = inspect_spatial_columns[column].notna().sum()
    print(f"{column}: {non_nan_count} non-NaN values")

# Display the first few rows of the selected columns
print("\nFirst 5 rows of selected spatial columns:")
print(inspect_spatial_columns.head())


Non-NaN value counts for selected spatial columns:
Lat Long: 5279 non-NaN values
Census Tract: 11554 non-NaN values
County: 16822 non-NaN values

First 5 rows of selected spatial columns:
                                     Lat Long  Census Tract      County
15688                      36.18133,-118.3087           NaN      Tulare
15689  40.36507,-120.95021;40.2403,-120.91051           NaN      Lassen
15690  38.05462,-121.71795;38.0414,-121.77348           NaN  Sacramento
15691                     39.30251,-120.11346           NaN      Placer
15692                      40.3321,-121.47321           NaN      Tehama


## Drop all rows without valid Census Tract

In [10]:
# Filter rows with NaN in 'Census Tract' but non-NaN in 'County'
dropped_na_rows_with_county = climate_mitigation_cci_data[
    climate_mitigation_cci_data['Census Tract'].isna() & climate_mitigation_cci_data['County'].notna()
]

# Drop rows with NaN in 'Census Tract' from the original DataFrame
climate_mitigation_cci_data = climate_mitigation_cci_data.dropna(subset=['Census Tract'])


In [12]:
# GEOID handling
climate_mitigation_cci_data['GEOID'] = climate_mitigation_cci_data['Census Tract'].apply(lambda x: '0' + str(x))
climate_mitigation_cci_data['GEOID'] = climate_mitigation_cci_data['GEOID'].astype(str).apply(lambda x: x.rstrip('0').rstrip('.') if '.' in x else x)

# Ensure all values in GEOID are strings
climate_mitigation_cci_data['GEOID'] = climate_mitigation_cci_data['GEOID'].astype(str)

In [13]:
climate_mitigation_cci_data

keep_columns = [
    'GEOID',
    'Total Project Cost',
    'Project Count',
    'SECTOR', 
    'CATEGORY', 
    'ACTION', 
    'Project Type',
    'Project Description',
    'Sub Program Name',
    'Other Project Benefits Description', 
    'Disadvantaged Community Benefits Description',
    'Is Low Income Communities',
    'Climate  Adaptation', 
    'Climate_Risk_Mitigation'
]

subset_climate_mitigation_cci = climate_mitigation_cci_data[keep_columns]
print(len(subset_climate_mitigation_cci))
subset_climate_mitigation_cci.head()

11554


,GEOID,Total Project Cost,Project Count,SECTOR,CATEGORY,ACTION,Project Type,Project Description,Sub Program Name,Other Project Benefits Description,Disadvantaged Community Benefits Description,Is Low Income Communities,Climate Adaptation,Climate_Risk_Mitigation
25097,06001401700,5563.0,1.0,Renewable Energy and Energy Efficiency,Energy Efficiency Upgrades and Rooftop Solar,Residential Installation,NaN,Provides single-family and small multi-family ...,Single-Family Energy Efficiency and Solar PV,"Energy savings, employment, training, improved...",Project provides energy efficiency upgrades an...,1.0,NaN,extreme heat mitigation
25098,06001402500,12237.0,3.0,Renewable Energy and Energy Efficiency,Energy Efficiency Upgrades and Rooftop Solar,Residential Installation,NaN,Provides single-family and small multi-family ...,Single-Family Energy Efficiency and Solar PV,"Energy savings, employment, training, improved...",Project provides energy efficiency upgrades an...,1.0,NaN,extreme heat mitigation
25099,06001406000,8584.0,1.0,Renewable Energy and Energy Efficiency,Energy Efficiency Upgrades and Rooftop Solar,Residential Installation,NaN,Provides single-family and small multi-family ...,Single-Family Energy Efficiency and Solar PV,"Energy savings, employment, training, improved...",Project provides energy efficiency upgrades an...,1.0,NaN,extreme heat mitigation
25100,06001406100,18124.0,2.0,Renewable Energy and Energy Efficiency,Energy Efficiency Upgrades and Rooftop Solar,Residential Installation,NaN,Provides single-family and small multi-family ...,Single-Family Energy Efficiency and Solar PV,"Energy savings, employment, training, improved...",Project provides energy efficiency upgrades an...,1.0,NaN,extreme heat mitigation
25101,06001407200,5028.0,1.0,Renewable Energy and Energy Efficiency,Energy Efficiency Upgrades and Rooftop Solar,Residential Installation,NaN,Provides single-family and small multi-family ...,Single-Family Energy Efficiency and Solar PV,"Energy savings, employment, training, improved...",Project provides energy efficiency upgrades an...,1.0,NaN,extreme heat mitigation


## Cal-CRAI

In [14]:
# GEOID handling
calcrai_score['GEOID'] = calcrai_score['GEOID'].apply(lambda x: '0' + str(x))
calcrai_score['GEOID'] = calcrai_score['GEOID'].astype(str).apply(lambda x: x.rstrip('0').rstrip('.') if '.' in x else x)

# Ensure all values in GEOID are strings
calcrai_score['GEOID'] = calcrai_score['GEOID'].astype(str)

calcrai_score = pd.merge(ca_tract_county, calcrai_score, on='GEOID', how='right')

keep_columns = [
    'GEOID',
    'County',
    'hazard_score',
    'calcrai_score'
]

subset_calcrai_score = calcrai_score[keep_columns]

## Merge CCI data (that have valid Climate Risk Mitigation entries) with Cal-CRAI
* Drop any rows that dont have Cal-CRAI score

In [16]:
cci_calcrai_merged = pd.merge(subset_climate_mitigation_cci, subset_calcrai_score, on='GEOID', how='left')
cci_calcrai_merged_hazard_calcrai = cci_calcrai_merged.dropna(subset=['calcrai_score'])
cci_calcrai_merged_hazard_calcrai.columns

Index(['GEOID', 'Total Project Cost', 'Project Count', 'SECTOR', 'CATEGORY',
       'ACTION', 'Project Type', 'Project Description', 'Sub Program Name',
       'Other Project Benefits Description',
       'Disadvantaged Community Benefits Description',
       'Is Low Income Communities', 'Climate  Adaptation',
       'Climate_Risk_Mitigation', 'County', 'hazard_score', 'calcrai_score'],
      dtype='object')

In [17]:
# Group by 'GEOID' and count the occurrences
geoid_counts = cci_calcrai_merged_hazard_calcrai.groupby('GEOID').size().reset_index(name='Count')

# Now, aggregate the 'hazard_score' and 'calcrai_score' columns by 'GEOID'
aggregated_scores = cci_calcrai_merged_hazard_calcrai.groupby('GEOID')[['hazard_score', 'calcrai_score']].mean().reset_index()

# Merge the counts with the aggregated scores
geoid_counts = geoid_counts.merge(aggregated_scores, on='GEOID')

# Sort in descending order based on 'Count'
geoid_counts = geoid_counts.sort_values(by='Count', ascending=False)

# Display the resulting DataFrame
print('Number of unique census tracts after merging CCI Climate Mitigation & Cal-CRAI (non nan climate mitigation or Cal-CRAI score columns):', 
      len(geoid_counts))
print('')
print('Median value of Cal-CRAI Score within subset (the higher, the more community capacity and overall resiliency):',geoid_counts.calcrai_score.median())
print('')
print('Median value of Climate Hazard Score within subset (the higher, the more at risk, 0-1 scale):',geoid_counts.hazard_score.median())
print('')
print(geoid_counts.head(5))

Number of unique census tracts after merging CCI Climate Mitigation & Cal-CRAI (non nan climate mitigation or Cal-CRAI score columns): 2731

Median value of Cal-CRAI Score within subset (the higher, the more community capacity and overall resiliency): 0.9185559004450764

Median value of Climate Hazard Score within subset (the higher, the more at risk, 0-1 scale): 0.42953871994077614

            GEOID  Count  hazard_score  calcrai_score
1172  06037543322     64      0.486640       1.844328
1170  06037543306     42      0.401251       3.192024
1168  06037543304     39      0.485234       1.704851
931   06037461902     39      0.348328       2.189421
1181  06037543801     32      0.533652       1.444490


In [18]:
# Count the number of entries for each unique GEOID and sort in descending order
geoid_counts = cci_calcrai_merged_hazard_calcrai.groupby('GEOID').size().reset_index(name='Count').sort_values(by='Count', ascending=False)

# Display the resulting DataFrame
print(geoid_counts.head())


            GEOID  Count
1172  06037543322     64
1170  06037543306     42
1168  06037543304     39
931   06037461902     39
1181  06037543801     32


In [22]:
cci_calcrai_merged_hazard_calcrai

,GEOID,Total Project Cost,Project Count,SECTOR,CATEGORY,ACTION,Project Type,Project Description,Sub Program Name,Other Project Benefits Description,Disadvantaged Community Benefits Description,Is Low Income Communities,Climate Adaptation,Climate_Risk_Mitigation,County,hazard_score,calcrai_score
0,06001401700,5563.0,1.0,Renewable Energy and Energy Efficiency,Energy Efficiency Upgrades and Rooftop Solar,Residential Installation,NaN,Provides single-family and small multi-family ...,Single-Family Energy Efficiency and Solar PV,"Energy savings, employment, training, improved...",Project provides energy efficiency upgrades an...,1.0,NaN,extreme heat mitigation,Alameda,0.686870,0.382663
1,06001402500,12237.0,3.0,Renewable Energy and Energy Efficiency,Energy Efficiency Upgrades and Rooftop Solar,Residential Installation,NaN,Provides single-family and small multi-family ...,Single-Family Energy Efficiency and Solar PV,"Energy savings, employment, training, improved...",Project provides energy efficiency upgrades an...,1.0,NaN,extreme heat mitigation,Alameda,0.662431,0.364412
2,06001406000,8584.0,1.0,Renewable Energy and Energy Efficiency,Energy Efficiency Upgrades and Rooftop Solar,Residential Installation,NaN,Provides single-family and small multi-family ...,Single-Family Energy Efficiency and Solar PV,"Energy savings, employment, training, improved...",Project provides energy efficiency upgrades an...,1.0,NaN,extreme heat mitigation,Alameda,0.688563,0.351280
3,06001406100,18124.0,2.0,Renewable Energy and Energy Efficiency,Energy Efficiency Upgrades and Rooftop Solar,Residential Installation,NaN,Provides single-family and small multi-family ...,Single-Family Energy Efficiency and Solar PV,"Energy savings, employment, training, improved...",Project provides energy efficiency upgrades an...,1.0,NaN,extreme heat mitigation,Alameda,0.839269,0.316310
4,06001407200,5028.0,1.0,Renewable Energy and Energy Efficiency,Energy Efficiency Upgrades and Rooftop Solar,Residential Installation,NaN,Provides single-family and small multi-family ...,Single-Family Energy Efficiency and Solar PV,"Energy savings, employment, training, improved...",Project provides energy efficiency upgrades an...,1.0,NaN,extreme heat mitigation,Alameda,0.753209,0.351219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11549,06037461400,1366.0,1.0,Water and Energy Efficiency,Water Efficiency,Incentives,High Efficiency Clothes Washer Commercial All-...,The project will expand the current WeDIP by o...,Water-Energy Grant Program,NaN,NaN,0.0,NaN,drought mitigation,Los Angeles,0.400334,2.161602
11550,06037462400,1920.0,8.0,Water and Energy Efficiency,Water Efficiency,Incentives,Lavatory Faucets,The project will expand the current WeDIP by o...,Water-Energy Grant Program,NaN,NaN,0.0,NaN,drought mitigation,Los Angeles,0.398406,2.057077
11551,06037461902,1366.0,1.0,Water and Energy Efficiency,Water Efficiency,Incentives,High Efficiency Clothes Washer Commercial All-...,The project will expand the current WeDIP by o...,Water-Energy Grant Program,NaN,NaN,0.0,NaN,drought mitigation,Los Angeles,0.348328,2.189421
11552,06037463500,1366.0,1.0,Water and Energy Efficiency,Water Efficiency,Incentives,High Efficiency Clothes Washer Commercial All-...,The project will expand the current WeDIP by o...,Water-Energy Grant Program,NaN,NaN,0.0,NaN,drought mitigation,Los Angeles,0.396480,2.126560


In [19]:
dropped_na_rows_with_county

,Project IDNumber,Reporting Cycle Name,Agency Name,Program Name,Program Description,Sub Program Name,Record Type,Project Name,Project Type,Project Description,...,Intermediary Admin Expenses Calc,PRIMARY_FUNDING_RECIPIENT_TYPE,TRIBAL AFFILIATION,PROJECT PARTNERS,Indicator,Detected_Metric_Keyword,Columns_Detected,Climate_Risk_Mitigation,Detected_Climate_Risk_Mitigation_Keyword,Columns_Detected_Climate_Risk
15688,P1496001,2015,California Department of Fish and Wildlife,Wetlands and Watershed Restoration,The Wetlands Restoration for Greenhouse Gas Re...,Wetlands and Watershed Restoration,AWARD-IMP,Developing a Protocol for Net Carbon Sequestra...,Mountain Meadow,Restore 90 acres of Osa Meadow using the pond ...,...,0.0,NaN,NaN,NaN,"Natural resources conservation, Inland floodin...","watershed, water quality, floodplain","Sub Program Name, Other Project Benefits Descr...","sea level rise mitigation, inland flooding mit...","wetland, floodplain","SECTOR, Other Project Benefits Description"
15689,P1496002,2015,California Department of Fish and Wildlife,Wetlands and Watershed Restoration,The Wetlands Restoration for Greenhouse Gas Re...,Wetlands and Watershed Restoration,AWARD-IMP,Mountain Meadows Restoration Project at Greenv...,Mountain Meadow,Restore 253 acres of degraded dry mountain mea...,...,0.0,NaN,NaN,NaN,"Natural resources conservation, Community prep...","community, watershed, water quality","Sub Program Name, Other Project Benefits Descr...","sea level rise mitigation, inland flooding mit...","wetland, riparian","SECTOR, Other Project Benefits Description"
15690,P1496003,2015,California Department of Fish and Wildlife,Wetlands and Watershed Restoration,The Wetlands Restoration for Greenhouse Gas Re...,Wetlands and Watershed Restoration,AWARD-IMP,Sherman Island Wetland Restoration Project (2 ...,Delta Wetland,Construction of a 700 ac Whale's Mouth Wetland...,...,0.0,NaN,NaN,NaN,"Natural resources conservation, Vulnerable pop...","construction, watershed, water quality, native","Sub Program Name, Other Project Benefits Descr...","sea level rise mitigation, inland flooding mit...","wetland, coastal, delta","SECTOR, Project Description, Project Type, CAT..."
15691,P1496004,2015,California Department of Fish and Wildlife,Wetlands and Watershed Restoration,The Wetlands Restoration for Greenhouse Gas Re...,Wetlands and Watershed Restoration,AWARD-IMP,Middle Martis Creek Wetlands Restoration,Mountain Meadow,Restore up to 37 acres of meadow and 2 acres o...,...,0.0,NaN,NaN,NaN,"Natural resources conservation, Social Services","construction, watershed, water quality","Sub Program Name, Project Description, Program...","sea level rise mitigation, inland flooding mit...","water storage, wetland, riparian","SECTOR, Project Description, Other Project Ben..."
15692,P1496005,2015,California Department of Fish and Wildlife,Wetlands and Watershed Restoration,The Wetlands Restoration for Greenhouse Gas Re...,Wetlands and Watershed Restoration,AWARD-IMP,A Demonstration of the Carbon Sequestration an...,Mountain Meadow,Restore 80 acres of Childs Meadow using cost-e...,...,0.0,NaN,NaN,NaN,Natural resources conservation,"watershed, water quality","Sub Program Name, Program Description","sea level rise mitigation, inland flooding mit...","wetland, riparian","SECTOR, Project Description"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133691,31,2023 Mid-Year Update,San Francisco Bay Conservation and Development...,Climate Resilience Planning,Works with local governments on adaptation pla...,Climate Change Adaptation and Coastal Resilien...,AWARD-IMP,Shoreline Development Projects with an Adaptio...,State Operations,BCDC's Regulatory staff conducts comprehensive...,...,0.0,NaN,NaN,NaN,Sea level rise exposure,"sea level rise, sea-level rise","Project Description, Program Description, CATE...",sea level rise mitigation,"sea level rise, coastal, sea-level rise, shore...","Sub Program Name, Project Description, 

In [21]:
keep_columns = [
    'County',
    'Total Project Cost',
    'Project Count',
    'SECTOR', 
    'CATEGORY', 
    'ACTION', 
    'Project Type',
    'Project Description',
    'Sub Program Name',
    'Other Project Benefits Description', 
    'Disadvantaged Community Benefits Description',
    'Is Low Income Communities',
    'Climate  Adaptation', 
    'Climate_Risk_Mitigation'
]

subset_county_climate_mitigation_rows = dropped_na_rows_with_county[keep_columns]
print(len(subset_county_climate_mitigation_rows))
subset_county_climate_mitigation_rows.head()

5268


,County,Total Project Cost,Project Count,SECTOR,CATEGORY,ACTION,Project Type,Project Description,Sub Program Name,Other Project Benefits Description,Disadvantaged Community Benefits Description,Is Low Income Communities,Climate Adaptation,Climate_Risk_Mitigation
15688,Tulare,1334646.0,1.0,Wetland Restoration and Management,Inland and Mountain Meadow,Restoration,Mountain Meadow,Restore 90 acres of Osa Meadow using the pond ...,Wetlands and Watershed Restoration,Restore habitat for mountain yellow-legged fro...,NaN,0.0,NaN,"sea level rise mitigation, inland flooding mit..."
15689,Lassen,699248.0,1.0,Wetland Restoration and Management,Inland and Mountain Meadow,Restoration,Mountain Meadow,Restore 253 acres of degraded dry mountain mea...,Wetlands and Watershed Restoration,Improve groundwater levels. Improve and create...,NaN,0.0,NaN,"sea level rise mitigation, inland flooding mit..."
15690,Sacramento,17542271.0,1.0,Wetland Restoration and Management,Coastal and Delta,Restoration,Delta Wetland,Construction of a 700 ac Whale's Mouth Wetland...,Wetlands and Watershed Restoration,Improves water quality and wildlife habitat. I...,Improves water quality and wildlife habitat. I...,0.0,NaN,"sea level rise mitigation, inland flooding mit..."
15691,Placer,1566065.0,1.0,Wetland Restoration and Management,Inland and Mountain Meadow,Restoration,Mountain Meadow,Restore up to 37 acres of meadow and 2 acres o...,Wetlands and Watershed Restoration,Increase water storage in a degraded meadow sy...,NaN,0.0,NaN,"sea level rise mitigation, inland flooding mit..."
15692,Tehama,758456.0,1.0,Wetland Restoration and Management,Inland and Mountain Meadow,Restoration,Mountain Meadow,Restore 80 acres of Childs Meadow using cost-e...,Wetlands and Watershed Restoration,Increase groundwater levels. Increase habitat ...,NaN,0.0,NaN,"sea level rise mitigation, inland flooding mit..."


In [34]:
# Group by 'GEOID' and count the occurrences
geoid_counts = cci_calcrai_merged_hazard_calcrai.groupby(['GEOID', 'County']).size().reset_index(name='Count')

# Aggregate the 'hazard_score' and 'calcrai_score' columns by 'GEOID' and 'county'
aggregated_scores = cci_calcrai_merged_hazard_calcrai.groupby(['GEOID', 'County'])[['hazard_score', 'calcrai_score']].mean().reset_index()

# Merge the counts with the aggregated scores
geoid_counts = geoid_counts.merge(aggregated_scores, on=['GEOID', 'County'])

# Sort in descending order based on 'Count'
geoid_counts = geoid_counts.sort_values(by='Count', ascending=False)

# Display the resulting DataFrame
print('Number of unique census tracts after merging CCI Climate Mitigation & Cal-CRAI (non nan climate mitigation or Cal-CRAI score columns):', 
      len(geoid_counts))
print('')
print('Median value of Cal-CRAI Score within subset (the higher, the more community capacity and overall resiliency):', geoid_counts['calcrai_score'].median())
print('')
print('Median value of Climate Hazard Score within subset (the higher, the more at risk, 0-1 scale):', geoid_counts['hazard_score'].median())
print('')
print(geoid_counts.head(50))


Number of unique census tracts after merging CCI Climate Mitigation & Cal-CRAI (non nan climate mitigation or Cal-CRAI score columns): 2731

Median value of Cal-CRAI Score within subset (the higher, the more community capacity and overall resiliency): 0.9185559004450764

Median value of Climate Hazard Score within subset (the higher, the more at risk, 0-1 scale): 0.42953871994077614

            GEOID       County  Count  hazard_score  calcrai_score
1172  06037543322  Los Angeles     64      0.486640       1.844328
1170  06037543306  Los Angeles     42      0.401251       3.192024
1168  06037543304  Los Angeles     39      0.485234       1.704851
931   06037461902  Los Angeles     39      0.348328       2.189421
1181  06037543801  Los Angeles     32      0.533652       1.444490
1336  06037602700  Los Angeles     29      0.485170       1.772201
934   06037462100  Los Angeles     29      0.400354       2.062663
922   06037460900  Los Angeles     26      0.401174       1.940916
1302  0603

In [39]:
subset_county_climate_mitigation_rows

,County,Total Project Cost,Project Count,SECTOR,CATEGORY,ACTION,Project Type,Project Description,Sub Program Name,Other Project Benefits Description,Disadvantaged Community Benefits Description,Is Low Income Communities,Climate Adaptation,Climate_Risk_Mitigation
15688,Tulare,1334646.0,1.0,Wetland Restoration and Management,Inland and Mountain Meadow,Restoration,Mountain Meadow,Restore 90 acres of Osa Meadow using the pond ...,Wetlands and Watershed Restoration,Restore habitat for mountain yellow-legged fro...,NaN,0.0,NaN,"sea level rise mitigation, inland flooding mit..."
15689,Lassen,699248.0,1.0,Wetland Restoration and Management,Inland and Mountain Meadow,Restoration,Mountain Meadow,Restore 253 acres of degraded dry mountain mea...,Wetlands and Watershed Restoration,Improve groundwater levels. Improve and create...,NaN,0.0,NaN,"sea level rise mitigation, inland flooding mit..."
15690,Sacramento,17542271.0,1.0,Wetland Restoration and Management,Coastal and Delta,Restoration,Delta Wetland,Construction of a 700 ac Whale's Mouth Wetland...,Wetlands and Watershed Restoration,Improves water quality and wildlife habitat. I...,Improves water quality and wildlife habitat. I...,0.0,NaN,"sea level rise mitigation, inland flooding mit..."
15691,Placer,1566065.0,1.0,Wetland Restoration and Management,Inland and Mountain Meadow,Restoration,Mountain Meadow,Restore up to 37 acres of meadow and 2 acres o...,Wetlands and Watershed Restoration,Increase water storage in a degraded meadow sy...,NaN,0.0,NaN,"sea level rise mitigation, inland flooding mit..."
15692,Tehama,758456.0,1.0,Wetland Restoration and Management,Inland and Mountain Meadow,Restoration,Mountain Meadow,Restore 80 acres of Childs Meadow using cost-e...,Wetlands and Watershed Restoration,Increase groundwater levels. Increase habitat ...,NaN,0.0,NaN,"sea level rise mitigation, inland flooding mit..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133691,San Francisco,132518.0,1.0,Climate Adaptation and Resilience,Sea Level Rise,Permitting,State Operations,BCDC's Regulatory staff conducts comprehensive...,Climate Change Adaptation and Coastal Resilien...,Provides climate resilience,No,0.0,NaN,sea level rise mitigation
133692,Contra Costa,46519.0,1.0,Active Transportation,Infrastructure,Planning,State Operations,BCDC's Regulatory staff conducts comprehensive...,Climate Change Adaptation and Coastal Resilien...,Provides climate resilience,No,0.0,NaN,sea level rise mitigation
133693,Solano,45405.0,1.0,Climate Adaptation and Resilience,Wetland Restoration and Management,Permitting,State Operations,BCDC's Regulatory staff conducts comprehensive...,Climate Change Adaptation and Coastal Resilien...,Provides climate resilience,No,0.0,NaN,sea level rise mitigation
133694,San Francisco,4698.0,1.0,Climate Adaptation and Resilience,Sea Level Rise,Capacity-Building,State Operations,BCDC improves expertise on climate change issu...,Climate Change Adaptation and Coastal Resilien...,Provides climate resilience,No,0.0,NaN,sea level rise mitigation


In [41]:
avg_county_hazard_index_scores = subset_calcrai_score.groupby(['County'])[['hazard_score', 'calcrai_score']].mean().reset_index()


In [43]:
cci_calcrai_merged_county = pd.merge(subset_county_climate_mitigation_rows, avg_county_hazard_index_scores, on='County', how='left')
cci_calcrai_merged_county = cci_calcrai_merged_county.dropna(subset=['calcrai_score'])
cci_calcrai_merged_county.columns

Index(['County', 'Total Project Cost', 'Project Count', 'SECTOR', 'CATEGORY',
       'ACTION', 'Project Type', 'Project Description', 'Sub Program Name',
       'Other Project Benefits Description',
       'Disadvantaged Community Benefits Description',
       'Is Low Income Communities', 'Climate  Adaptation',
       'Climate_Risk_Mitigation', 'hazard_score', 'calcrai_score'],
      dtype='object')

In [50]:
# Group by 'GEOID' and count the occurrences
county_counts = cci_calcrai_merged_county.groupby(['County']).size().reset_index(name='Count')

# Aggregate the 'hazard_score' and 'calcrai_score' columns by 'GEOID' and 'county'
aggregated_scores = cci_calcrai_merged_county.groupby(['County'])[['hazard_score', 'calcrai_score']].mean().reset_index()

# Merge the counts with the aggregated scores
county_counts = county_counts.merge(aggregated_scores, on=['County'])

# Sort in descending order based on 'Count'
county_counts = county_counts.sort_values(by='Count', ascending=False)

county_counts = county_counts.rename(columns={'hazard_score':'county_hazard_score_average', 'calcrai_score':'county_calcrai_score_average'})

# Display the resulting DataFrame
print('Number of unique CA counties after merging CCI Climate Mitigation & Cal-CRAI (non nan climate mitigation or Cal-CRAI score columns):', 
      len(county_counts))
print('')
print('Median value of Cal-CRAI Score within subset (the higher, the more community capacity and overall resiliency):', county_counts['county_calcrai_score_average'].median())
print('')
print('Median value of Climate Hazard Score within subset (the higher, the more at risk, 0-1 scale):', county_counts['county_hazard_score_average'].median())
print('')
print(county_counts.head(5))


Number of unique CA counties after merging CCI Climate Mitigation & Cal-CRAI (non nan climate mitigation or Cal-CRAI score columns): 58

Median value of Cal-CRAI Score within subset (the higher, the more community capacity and overall resiliency): 0.7000408844462012

Median value of Climate Hazard Score within subset (the higher, the more at risk, 0-1 scale): 0.4086448643325852

         County  Count  county_hazard_score_average  \
18  Los Angeles    453                     0.412929   
9        Fresno    381                     0.253079   
53       Tulare    293                     0.337160   
36    San Diego    237                     0.679150   
33   Sacramento    214                     0.691559   

    county_calcrai_score_average  
18                      1.976216  
9                       0.896718  
53                      0.470513  
36                      0.628036  
33                      0.597499  
